In [1]:
# basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

# sklearn
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# others
# import multiprocessing as mp
from xgboost import XGBRegressor
from tqdm import tqdm

# config
import sys
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/writeup')
from config import C_COMMON, C_SINGLE, C_GRID, FILENAME
from util import load_data, XGB, NeuralNetwork, LinearRegression

# enable autoreload
%load_ext autoreload
%autoreload 2

2022-08-12 01:11:58.165796: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
columns = C_SINGLE
df_train, df_test = load_data(columns + C_COMMON, FILENAME)

# Model performance comparison

In [7]:
l_model = LinearRegression(columns=columns)
l_pred = l_model.evaluate(df_train, df_test)

,skn,rmse_lr,mae_lr
0,54.0,4.833861,3.558593
1,79.0,6.301111,4.208636
2,338.0,2.818706,2.123845
3,250.0,2.189325,1.443336
4,267.0,2.243758,1.505239
5,296.1,1.250883,0.888049
6,311.0,1.188327,0.864515
7,396.0,1.308114,0.960739
8,400.0,1.298378,0.976187
9,406.0,1.643493,1.254386


In [48]:
# XGBoost
x_model = XGB(
    params={
        'n_estimators': 120,
        'learning_rate': 0.070755,
        'max_depth': 2
    },
    columns=columns
)
x_pred = x_model.evaluate(df_train, df_test)
x_pred

,skn,rmse_xgb,mae_xgb
0,54.0,5.020365,3.724248
1,79.0,6.714039,4.751905
2,338.0,2.948576,2.119263
3,250.0,2.196315,1.437160
4,267.0,2.200451,1.434376
5,296.1,1.222167,0.723007
6,311.0,1.181025,0.671441
7,396.0,1.235662,0.781160
8,400.0,1.260604,0.864615
9,406.0,1.600808,1.175606


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import L2

def define_model(
    input_dim=20,
    n_units=512,
    activation='selu',#selu
    learning_rate=0.00001,
    loss='mse',
    batch_size=64
):
    inputs = Input(shape=(input_dim))
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(inputs)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=n_units, activation=activation, kernel_regularizer=L2(l2=0.01))(x)
    x = Dropout(rate=0.5)(x)# serves as regularization
    outputs = Dense(units=1, activation='softplus')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=tf.optimizers.Adam(learning_rate=learning_rate),
        loss=loss,
        metrics=[RootMeanSquaredError()]
    )
    return model, batch_size
n_model = NeuralNetwork(
    params={
        'n_units': 559,
        'learning_rate': 0.0004886843292039,
        'input_dim': 16,
        'batch_size': 256,
        'loss': 'mse'
    },
    columns=columns,
    model_func=define_model
)

In [4]:
n_pred = n_model.evaluate(df_train, df_test)

  0%|          | 0/24 [00:00<?, ?it/s]2022-08-09 01:29:15.338935: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-09 01:29:15.340828: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-09 01:29:15.410430: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: NVIDIA GeForce RTX 2070 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 36 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-08-09 01:29:15.410487: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-09 01:29:15.413362: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-08-09 01:29:15.413456: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully op

In [8]:
n_pred.merge(l_pred, left_on='skn', right_on='skn')

,skn,rmse_nn,mae_nn,rmse_lr,mae_lr
0,54.0,4.706835,3.156515,4.833861,3.558593
1,79.0,6.241887,3.912455,6.301111,4.208636
2,338.0,2.697565,1.827145,2.818706,2.123845
3,250.0,2.322117,1.331094,2.189325,1.443336
4,267.0,2.350901,1.443302,2.243758,1.505239
5,296.1,1.120728,0.629470,1.250883,0.888049
6,311.0,1.184260,0.581202,1.188327,0.864515
7,396.0,1.267793,0.740079,1.308114,0.960739
8,400.0,1.367699,0.880745,1.298378,0.976187
9,406.0,1.648682,1.175153,1.643493,1.254386


In [10]:
n_pred['rmse_nn'].mean(), l_pred['rmse_lr'].mean()

(2.7220337888275448, 2.811234342108683)

# site specific hyperparameter for NN

In [2]:
from hyperparameters import NN_PARAMS

In [23]:
ret_vals = []
for item in tqdm(NN_PARAMS):
    station_model = NeuralNetwork(
        columns=columns,
        params=item['params'],
        model_func=define_model
    )
    r = station_model.evaluate_by_station(df_train, df_test, skn=item['skn'])
    ret_vals.append(r)

100%|██████████| 24/24 [11:09<00:00, 27.88s/it]


In [25]:
temp = pd.DataFrame(ret_vals).merge(l_pred, left_on='skn', right_on='skn')
temp['rmse_nn'].mean(), temp['rmse_lr'].mean()

(2.7416193234218524, 2.8112343421086834)

# try data generator

In [3]:
skn = 54
df_train_station = df_train[skn == 54]
df_test_station = 

,skn,year,month,data_in,lat,lon,air2m,air1000_500,hgt500,hgt1000,...,shum700,shum925,skt,slp,elevation,season_wet,fold,len,cumsum,inner_fold
0,54.0,1948,1,5.370000,19.429741,-155.257146,295.39603,31.299995,5799.5483,121.483870,...,2.945999,9.869999,23.385218,1014.08490,3970.00,1,0,21,21,0
1,79.0,1948,1,6.170000,19.548635,-155.300557,295.39603,31.299995,5799.5483,121.483870,...,2.945999,9.869999,23.385218,1014.08490,5170.00,1,0,21,21,0
2,338.0,1948,1,30.890000,20.758467,-156.245523,295.39603,31.299995,5799.5483,121.483870,...,2.945999,9.869999,23.385218,1014.08490,7030.00,1,0,21,21,0
3,250.0,1948,1,5.270000,20.648174,-156.398497,295.30087,31.379990,5799.2256,121.258064,...,2.826000,9.923000,23.754120,1013.94574,1900.00,1,0,21,21,0
4,267.0,1948,1,7.800000,20.700140,-156.355519,295.30087,31.379990,5799.2256,121.258064,...,2.826000,9.923000,23.754120,1013.94574,3005.00,1,0,21,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14757,1020.1,1999,11,2.350000,21.981570,-159.342206,297.49582,30.769989,5860.4670,155.966660,...,2.562000,11.216000,25.869417,1018.15094,103.00,1,3,24,14762,4
14758,1075.0,1999,11,1.860000,22.130187,-159.658870,297.49582,30.769989,5860.4670,155.966660,...,2.562000,11.216000,25.869417,1018.15094,3600.00,1,3,24,14762,4
14759,1117.0,1999,11,8.880000,22.213196,-159.472201,297.49582,30.769989,5860.4670,155.966660,...,2.562000,11.216000,25.869417,1018.15094,295.00,1,3,24,14762,4
14760,1134.0,1999,11,3.400000,22.211941,-159.407202,297.49582,30.769989,5860.4670,155.966660,...,2.562000,11.216000,25.869417,1018.15094,315.00,1,3,24,14762,4


# does the number of samples affect the performance?

In [5]:
columns = C_SINGLE
df_train, df_test = load_data(columns + C_COMMON, FILENAME)

In [8]:
percentage = 0.7
ret_vals_07 = [] 
for item in NN_PARAMS:
    df_train_station = df_train[df_train['skn'] == item['skn']]
    df_train_station = df_train_station[:int(df_train_station.shape[0] * percentage)]
    model = NeuralNetwork(
        columns=columns,
        params=item['params'],
        model_func=define_model
    )
    r = model.evaluate_by_station(df_train_station, df_test, skn=item['skn'])
    ret_vals_07.append(r)

2022-08-10 02:23:44.284895: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-10 02:23:44.286767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-10 02:23:44.330666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1c:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-08-10 02:23:44.330744: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-10 02:23:44.333395: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-08-10 02:23:44.333491: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

In [9]:
percentage = 0.8
ret_vals_08 = [] 
for item in NN_PARAMS:
    df_train_station = df_train[df_train['skn'] == item['skn']]
    df_train_station = df_train_station[:int(df_train_station.shape[0] * percentage)]
    model = NeuralNetwork(
        columns=columns,
        params=item['params'],
        model_func=define_model
    )
    r = model.evaluate_by_station(df_train_station, df_test, skn=item['skn'])
    ret_vals_08.append(r)

In [10]:
percentage = 0.9
ret_vals_09 = [] 
for item in NN_PARAMS:
    df_train_station = df_train[df_train['skn'] == item['skn']]
    df_train_station = df_train_station[:int(df_train_station.shape[0] * percentage)]
    model = NeuralNetwork(
        columns=columns,
        params=item['params'],
        model_func=define_model
    )
    r = model.evaluate_by_station(df_train_station, df_test, skn=item['skn'])
    ret_vals_09.append(r)

In [11]:
ret_vals_07

[{'skn': 54.0, 'rmse_nn': 4.697541267519356, 'mae_nn': 3.2408318502475053},
 {'skn': 79.0, 'rmse_nn': 6.482653398671262, 'mae_nn': 4.086567645316576},
 {'skn': 87.0, 'rmse_nn': 5.2637839062530904, 'mae_nn': 3.383921548333138},
 {'skn': 250.0, 'rmse_nn': 2.257535635270435, 'mae_nn': 1.325187365295021},
 {'skn': 267.0, 'rmse_nn': 2.2874748162280425, 'mae_nn': 1.411407293220619},
 {'skn': 296.1, 'rmse_nn': 1.1190836856523314, 'mae_nn': 0.6068030546440019},
 {'skn': 311.0, 'rmse_nn': 1.2352309713299998, 'mae_nn': 0.5978776363810155},
 {'skn': 338.0, 'rmse_nn': 2.7339825775941704, 'mae_nn': 1.8429897126228576},
 {'skn': 396.0, 'rmse_nn': 1.2889969800613938, 'mae_nn': 0.7221363949164366},
 {'skn': 400.0, 'rmse_nn': 1.3691488829534104, 'mae_nn': 0.8882542379757333},
 {'skn': 406.0, 'rmse_nn': 1.5936896515970573, 'mae_nn': 1.140953906551484},
 {'skn': 410.0, 'rmse_nn': 1.4570706121638475, 'mae_nn': 0.9351698936865879},
 {'skn': 485.0, 'rmse_nn': 1.8306103652220331, 'mae_nn': 1.394978325122442}

In [14]:
ret_vals_08

[{'skn': 54.0, 'rmse_nn': 4.7112584150003105, 'mae_nn': 3.155426157315571},
 {'skn': 79.0, 'rmse_nn': 6.2539804692455165, 'mae_nn': 4.061204986363432},
 {'skn': 87.0, 'rmse_nn': 5.286770761198661, 'mae_nn': 3.43232350616698},
 {'skn': 250.0, 'rmse_nn': 2.2527588558478495, 'mae_nn': 1.3278542890366476},
 {'skn': 267.0, 'rmse_nn': 2.2691309177256413, 'mae_nn': 1.4167298325006061},
 {'skn': 296.1, 'rmse_nn': 1.1452382297912633, 'mae_nn': 0.5998489150736066},
 {'skn': 311.0, 'rmse_nn': 1.2211481367067687, 'mae_nn': 0.5963151646267836},
 {'skn': 338.0, 'rmse_nn': 2.6825351233371837, 'mae_nn': 1.8312789003003027},
 {'skn': 396.0, 'rmse_nn': 1.3578178517209425, 'mae_nn': 0.7421286876385028},
 {'skn': 400.0, 'rmse_nn': 1.4412598373337941, 'mae_nn': 0.920616336855868},
 {'skn': 406.0, 'rmse_nn': 1.604992157528849, 'mae_nn': 1.1481649055173317},
 {'skn': 410.0, 'rmse_nn': 1.4954322450739483, 'mae_nn': 0.946935026309429},
 {'skn': 485.0, 'rmse_nn': 1.8600912053581817, 'mae_nn': 1.3991204719054393

In [13]:
ret_vals_09

[{'skn': 54.0, 'rmse_nn': 4.684726024342567, 'mae_nn': 3.119520556926726},
 {'skn': 79.0, 'rmse_nn': 6.429969028504741, 'mae_nn': 4.019784808611347},
 {'skn': 87.0, 'rmse_nn': 5.498968419153935, 'mae_nn': 3.9210810443246444},
 {'skn': 250.0, 'rmse_nn': 2.264055048832901, 'mae_nn': 1.3330430364305044},
 {'skn': 267.0, 'rmse_nn': 2.33922715055318, 'mae_nn': 1.4426002452280613},
 {'skn': 296.1, 'rmse_nn': 1.118012468748036, 'mae_nn': 0.6531763425469398},
 {'skn': 311.0, 'rmse_nn': 1.2456385340741871, 'mae_nn': 0.6044071876926786},
 {'skn': 338.0, 'rmse_nn': 2.7924412251864976, 'mae_nn': 1.8585534263426255},
 {'skn': 396.0, 'rmse_nn': 1.269163324375048, 'mae_nn': 0.7498567036482005},
 {'skn': 400.0, 'rmse_nn': 1.5153234238878093, 'mae_nn': 0.9572737298237056},
 {'skn': 406.0, 'rmse_nn': 1.6881060829238685, 'mae_nn': 1.2210033992029004},
 {'skn': 410.0, 'rmse_nn': 1.4561801912165104, 'mae_nn': 0.9354073692590762},
 {'skn': 485.0, 'rmse_nn': 1.877184236883337, 'mae_nn': 1.4822632704025658},
